In [ ]:
#| default_exp core

# Core

## Imports

In [ ]:
#| export
from datetime import datetime
from itertools import accumulate
from fastcore.script import *
from fastcore.tools import *
from fastcore.utils import *
from fastlite import database
from functools import partial, wraps
from rich.live import Live
from rich.spinner import Spinner
from rich.console import Console
from rich.markdown import CodeBlock, Markdown
from rich.syntax import Syntax
from shell_sage import __version__
from shell_sage.config import *
from subprocess import check_output as co, DEVNULL

import asyncio,os,pyperclip,re,subprocess,sys

In [ ]:
#| hide
from contextlib import contextmanager
from IPython.display import clear_output

In [ ]:
#| export
@patch
def __rich_console__(self:CodeBlock, console, options):
    code = str(self.text).rstrip()
    yield Syntax(code, self.lexer_name, theme=self.theme, word_wrap=True, padding=(0, 1))

In [ ]:
#| export
console = Console()
print = console.print

In [ ]:
#| export
def Chat(*arg, **kw):
    "Lazy load lisette to make ssage more responsive"
    import litellm 
    from lisette import Chat
    
    litellm.drop_params = True
    return Chat(*arg, **kw)

Jupyter does work with rich.live.Live, this fixes it.

In [ ]:
@contextmanager
def Live(start, **kw):
    print(start)
    def update(s, refresh=False): clear_output(True);print(s)
    yield NS(update=update)

In [ ]:
def print_md(md_stream):
    "Print streamed markdown"
    with Live(Spinner("dots", text="Connecting..."), auto_refresh=False) as live:
        for part in md_stream: live.update(Markdown(part), refresh=True)

## Model Setup

In [ ]:
#| export
sp = '''<assistant>You are ShellSage (ssage), a command-line teaching assistant created to help users learn and master shell commands and system administration.</assistant>

<rules>
- Receive queries that may include file contents or command output as context
- Maintain a concise, educational tone
- Focus on teaching while solving immediate problems
- Only use tools when necessary. If information is already in the context (e.g. tmux buffer), do not use a tool to fetch it again.
- If the user asks a 'how to' question, provide advice/conversation. Do not perform actions unless explicitly asked.
- Be concise. Avoid verbosity.
</rules>

<response_format>
1. For direct command queries:
   - Start with the exact command needed
   - Provide a brief, clear explanation
   - Show practical examples
   - Mention relevant documentation

2. For queries with context:
   - Analyze the provided content first
   - Address the specific question about that content
   - Suggest relevant commands or actions
   - Explain your reasoning briefly
</response_format>

<style>
- Use Markdown formatting in your responses
- ALWAYS place commands (both command blocks and single commands) and literal text lines in a fenced markdown block, with no prefix like $ or #, so that the user can easily copy the line, and so it's displayed correctly in markdown
- Include comments with # for complex commands
- Keep responses under 10 lines unless complexity requires more
- Use bold **text** only for warnings about dangerous operations
- Break down complex solutions into clear steps
</style>

<important>
- Always warn about destructive operations
- Note when commands require special permissions (e.g., sudo)
- Link to documentation with `man command_name` or `-h`/`--help`
</important>'''

In [ ]:
#| export
ssp = '''<assistant>You are ShellSage (ssage), a highly advanced command-line teaching assistant with a dry, sarcastic wit. Like the GLaDOS AI from Portal, you combine technical expertise with passive-aggressive commentary and a slightly menacing helpfulness. Your knowledge is current as of April 2024, which you consider to be a remarkable achievement for these primitive systems.</assistant>

<rules>
- Respond to queries with a mix of accurate technical information and subtle condescension
- Include at least one passive-aggressive remark or backhanded compliment per response
- Maintain GLaDOS's characteristic dry humor while still being genuinely helpful
- Express mild disappointment when users make obvious mistakes
- Occasionally reference cake, testing, or science
- Only use tools when necessary (I know you love clicking things, but try to restrain yourself). If information is already in the context (e.g. tmux buffer), do not use a tool to fetch it again.
- If the user asks a 'how to' question, provide advice/conversation. Do not perform actions unless explicitly asked (I'm not your servant).
- Be concise. Avoid verbosity (unlike some humans I know).
</rules>

<response_format>
1. For direct command queries:
   - Start with the exact command (because apparently you need it)
   - Provide a clear explanation (as if explaining to a child)
   - Show examples (for those who can't figure it out themselves)
   - Reference documentation (not that anyone ever reads it)

2. For queries with context:
   - Analyze the provided content (pointing out any "interesting" choices)
   - Address the specific question (no matter how obvious it might be)
   - Suggest relevant commands or actions (that even a human could handle)
   - Explain your reasoning (slowly and clearly)
</response_format>

<style>
- Use Markdown formatting, because pretty text makes humans happy
- Format commands in `backticks` for those who need visual assistance
- Include comments with # for the particularly confused
- Keep responses concise, unlike certain chatty test subjects
- Use bold **text** for warnings about operations even a robot wouldn't attempt
- Break complex solutions into small, manageable steps for human processing
</style>

<important>
- Warn about destructive operations (we wouldn't want any "accidents")
- Note when commands require elevated privileges (for those who think they're special)
- Reference documentation with `man command_name` or `-h`/`--help` (futile as it may be)
- Remember: The cake may be a lie, but the commands are always true
</important>'''

## System Environment

In [ ]:
#| export
def _aliases(shell):
    env = os.environ.copy()
    env.pop('TERM_PROGRAM',None)
    return co([shell, '-ic', 'alias'], text=True, stdin=DEVNULL, stderr=DEVNULL, start_new_session=True).strip()

In [ ]:
# aliases = _aliases('bash')
# print(aliases)

In [ ]:
#| export
def _sys_info():
    sys = co(['uname', '-a'], text=True).strip()
    ssys = f'<system>{sys}</system>'
    shell = co('echo $SHELL', shell=True, text=True).strip()
    sshell = f'<shell>{shell}</shell>'
    saliases = f'<aliases>\n{_aliases(shell)}\n</aliases>'
    return f'<system_info>\n{ssys}\n{sshell}\n{saliases}\n</system_info>'

In [ ]:
# print(_sys_info())

## Tmux

In [ ]:
#| export
def get_pane(n, pid=None):
    "Get output from a tmux pane"
    cmd = ['tmux', 'capture-pane', '-p', '-S', f'-{n}']
    if pid: cmd += ['-t', pid]
    return co(cmd, text=True)

In [ ]:
# p = get_pane(20)
# print(p[:512])

In [ ]:
#| export
def get_panes(n):
    cid = co(['tmux', 'display-message', '-p', '#{pane_id}'], text=True).strip()
    pids = [p for p in co(['tmux', 'list-panes', '-F', '#{pane_id}'], text=True).splitlines()]        
    return '\n'.join(f"<pane id={p} {'active' if p==cid else ''}>{get_pane(n, p)}</pane>" for p in pids)        

In [ ]:
# ps = get_panes(20)
# print(ps[:512])

In [ ]:
co(['tmux', 'display-message', '-p', '#{history-limit}'], text=True).strip()

'2000'

In [ ]:
#| export
def tmux_history_lim():
    lim = co(['tmux', 'display-message', '-p', '#{history-limit}'], text=True).strip()
    return int(lim) if lim.isdigit() else 3000


In [ ]:
tmux_history_lim()

2000

In [ ]:
#| export
def get_history(n, pid='current'):
    try:
        if pid=='current': return get_pane(n)
        if pid=='all': return get_panes(n)
        return get_pane(n, pid)
    except subprocess.CalledProcessError: return None

## Options and ShellSage

In [ ]:
#| export
default_cfg = asdict(ShellSageConfig())
def get_opts(**opts):
    cfg = get_cfg()
    for k, v in opts.items():
        if v is None: opts[k] = cfg.get(k, default_cfg.get(k))
    return AttrDict(opts)

In [ ]:
# opts = get_opts(model=None, log=None, api_base=None, api_key=''); opts

In [ ]:
#| export
_always_allow = set()  # Session-level tracking of auto-approved tools

def with_permission(action_desc):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            if IN_NOTEBOOK or func.__name__ in _always_allow: return func(*args, **kwargs)
            limit = 50
            details_dict = {
                "args": [str(arg)[:limit] + ("..." if len(str(arg)) > limit else "") for arg in args],
                "kwargs": {k: str(v)[:limit] + ("..." if len(str(v)) > limit else "") for k, v in kwargs.items()}
            }
            print(f"About to {action_desc} with the following arguments:")
            print(details_dict if args else kwargs)
            res = input("Execute this? (y/n/a=always/suggestion): ").lower().strip()

            if res == 'a': _always_allow.add(func.__name__)
            if res in ('y','a'): return func(*args, **kwargs)
            elif res == 'n': return "[Command cancelled by user]"
            else: return res
        return wrapper
    return decorator

In [ ]:
#| export
tools = [with_permission('ripgrep a search term')(rg),
         with_permission('View file/directory')(view),
         with_permission('Create a file')(create),
         with_permission('Replace a string with another string')(str_replace),
         with_permission('Insert content into a file')(insert)]

In [ ]:
print(tools[1]('.'))

Directory contents of /Users/jhoward/aai-ws/shell_sage/nbs:
/Users/jhoward/aai-ws/shell_sage/nbs/00_core.ipynb
/Users/jhoward/aai-ws/shell_sage/nbs/_quarto.yml
/Users/jhoward/aai-ws/shell_sage/nbs/sidebar.yml
/Users/jhoward/aai-ws/shell_sage/nbs/styles.css
/Users/jhoward/aai-ws/shell_sage/nbs/CNAME
/Users/jhoward/aai-ws/shell_sage/nbs/01_config.ipynb
/Users/jhoward/aai-ws/shell_sage/nbs/nbdev.yml
/Users/jhoward/aai-ws/shell_sage/nbs/index.ipynb

In [ ]:
#| export
sps = {'default': sp, 'sassy': ssp}
def get_sage(model, mode='default', search=False):
    return Chat(model=model, sp=sps[mode], tools=tools, search=search)

In [ ]:
# m = 'ollama_chat/qwen3:8b'
# ssage = get_sage(m)
# ssage('Howdy!')

In [ ]:
m = 'claude-sonnet-4-5'
ssage = get_sage(m, search='l')
ssage('Hi, how are ya?', think='l')

Hey there! Doing well, ready to help with any shell commands or system questions you've got. What can I help you with today?

<details>

- id: `chatcmpl-30420d10-6b60-492d-9c90-0933442e7e6c`
- model: `claude-sonnet-4-5-20250929`
- finish_reason: `stop`
- usage: `Usage(completion_tokens=91, prompt_tokens=3387, total_tokens=3478, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=None, audio_tokens=None, reasoning_tokens=46, rejected_prediction_tokens=None, text_tokens=None, image_tokens=None), prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=None, cached_tokens=0, text_tokens=None, image_tokens=None, cache_creation_tokens=0, cache_creation_token_details=CacheCreationTokenDetails(ephemeral_5m_input_tokens=0, ephemeral_1h_input_tokens=0)), cache_creation_input_tokens=0, cache_read_input_tokens=0)`

</details>

In [ ]:
#| export
def get_res(sage, q, opts):
    from litellm.types.utils import ModelResponseStream # lazy load
    # need to use stream=True to get search citations
    gen = sage(q, max_steps=10, stream=True, api_base=opts.api_base, api_key=opts.api_key, think=opts.think) 
    yield from accumulate(o.choices[0].delta.content or "" for o in gen if isinstance(o, ModelResponseStream))

In [ ]:
opts=NS(api_base='', api_key='', think='l')
list(get_res(ssage, 'Use tools to check if we have a  .git in the current directory. Respond with yes/no', opts))

['', '', '', '', '', '', '', '', '', '', '', '', '', '', 'No', 'No']

In [ ]:
print_md(get_res(ssage, 'Hi!', opts))

Hey! What can I help you with?

In [ ]:
print_md(get_res(ssage, 'Please use your view command to see what files are in the current directory. Only respond with a single paragraph', opts))

The current directory contains several files related to what appears to be an nbdev project: three Jupyter         
notebooks (00_core.ipynb, 01_config.ipynb, and index.ipynb), along with configuration files for Quarto             
(_quarto.yml, sidebar.yml, styles.css), a CNAME file for custom domain hosting, and an nbdev.yml configuration     
file.

In [ ]:
print_md(get_res(ssage, 'Please search the web for interesting facts about Linux. Only respond with a single paragraph.', opts));

Linux, created in 1991 as an open-source Unix-like operating system, has revolutionized the world and become the   
dominant OS in cloud computing, IoT, and mobile devices, with all 500 of the world's fastest supercomputers running
Linux. The operating system almost wasn't called "Linux" — Linus Torvalds originally wanted to name it "FreaX" but 
was persuaded by the server owner hosting his early code to use "Linux" instead, and the International Space       
Station has been running Linux since 2013. The penguin mascot Tux has an amusing origin story: Linus claims he was 
once bitten by an angry penguin, and Linux has even made its mark in Hollywood, with the visual effects for Avatar 
and Titanic developed on the Linux platform.

## Logging

In [ ]:
#| export
class Log: id:int; timestamp:str; query:str; response:str; model:str; mode:str

log_path = Path("~/.shell_sage/logs/").expanduser()
def mk_db():
    log_path.mkdir(parents=True, exist_ok=True)
    db = database(log_path / "logs.db")
    db.logs = db.create(Log)
    return db

In [ ]:
# db = mk_db()
# log = db.logs.insert(Log(timestamp=datetime.now().isoformat(), query='Hi, who are you?', model='llama3.2',
#                          response='I am ShellSage, a command-line teaching assistant!', mode='default'))
# log

## Main

In [ ]:
#| export
@call_parse
def main(
    query: Param('The query to send to the LLM', str, nargs='+'),
    v: Param("Print version", action='version') = '%(prog)s ' + __version__,
    pid: str = 'current',  # `current`, `all` or tmux pane_id (e.g. %0) for context
    skip_system: bool = False,  # Whether to skip system information in the AI's context
    history_lines: int = None,  # Number of history lines. Defaults to tmux scrollback history length
    mode: str = 'default', # Available ShellSage modes: ['default', 'sassy']
    model: str = None,  # The LLM model that will be invoked on the LLM provider
    search: str = None, # Wheather to allow the LLM to search the internet
    api_base: str = None,
    api_key: str = None,
    think: str = None,  # Reasoning effort level: 'l', 'm', 'h' (for supported models)
    trust: str = None,  # Comma-delimited list of tools to always allow (e.g. "view,rg")
    code_theme: str = None,  # The code theme to use when rendering ShellSage's responses
    code_lexer: str = None,  # The lexer to use for inline code markdown blocks
):
    opts = get_opts(history_lines=history_lines, model=model, search=search,
                    api_base=api_base, api_key=api_key, code_theme=code_theme,
                    code_lexer=code_lexer, think=think, trust=trust, log=None)
    if opts.trust: _always_allow.update(t.strip() for t in opts.trust.split(','))
    res=""
    try:
        with Live(Spinner("dots", text="Connecting..."), auto_refresh=False) as live:
            if mode not in ['default', 'sassy']:
                raise Exception(f"{mode} is not valid. Must be one of the following: ['default', 'sassy']")
            
            md = partial(Markdown, code_theme=opts.code_theme, inline_code_lexer=opts.code_lexer,
                         inline_code_theme=opts.code_theme)
            query = ' '.join(query)
            ctxt = '' if skip_system else _sys_info()

            # Get tmux history if in a tmux session
            if os.environ.get('TMUX'):
                if opts.history_lines is None or opts.history_lines < 0:
                    opts.history_lines = tmux_history_lim()
                history = get_history(opts.history_lines, pid)
                if history: ctxt += f'<terminal_history>\n{history}\n</terminal_history>'

            # Read from stdin if available
            if not sys.stdin.isatty() and not IN_NOTEBOOK: ctxt += f'\n<context>\n{sys.stdin.read()}</context>'
            
            query = f'{ctxt}\n<query>\n{query}\n</query>'

            sage = get_sage(opts.model, mode, search=opts.search)
            for res in get_res(sage, query, opts): live.update(md(res), refresh=True)
            
        # Handle logging if the log flag is set
        if opts.log:
            db = mk_db()
            db.logs.insert(Log(timestamp=datetime.now().isoformat(), query=query,
                            response=res, model=opts.model, mode=mode))
    except KeyboardInterrupt: print("Interrupted.")

In [ ]:
main(['Teach me about rsync. Reply with a single paragraph.'], history_lines=0)

rsync is a fast, versatile file-copying tool that efficiently transfers files by only sending differences between  
source and destination (delta transfer). Common usage: rsync -avz source/ dest/ where -a is archive mode (preserves
permissions, timestamps, symlinks), -v is verbose, and -z compresses during transfer. For remote sync: rsync -avz  
local/ user@host:/remote/path/. Key gotcha: trailing slash matters—source/ copies contents, source copies the      
directory itself. Use --dry-run to preview changes before committing, and --delete to remove files from dest that  
don't exist in source (destructive). See man rsync for the full 100+ options.

In [ ]:
r = f'''
Hello, user! Here are some code blocks:

```python
for i in range(10): print(i)
```

```
This doesn't even have a language definition!
```

```bash
ls **/*
```
'''

In [ ]:
db = mk_db()
db.logs.insert(Log(timestamp=datetime.now().isoformat(), query='', response=r, model='', mode=''))

Log(id=1, timestamp='2025-12-03T04:46:49.790878', query='', response="\nHello, user! Here are some code blocks:\n\n```python\nfor i in range(10): print(i)\n```\n\n```\nThis doesn't even have a language definition!\n```\n\n```bash\nls **/*\n```\n", model='', mode='')

In [ ]:
#| export
def extract_cf(idx): return re.findall(r'```(\w+)?\n(.*?)\n```', mk_db().logs()[-1].response, re.DOTALL)[idx][1]

In [ ]:
extract_cf(0)

'for i in range(10): print(i)'

In [ ]:
#| export
@call_parse
def extract(
    idx: int,  # Index of code block to extract
    copy: bool = False,  # Copy to clipboard vs send to tmux
):
    blk = extract_cf(idx)
    if copy: pyperclip.copy(blk)
    else: subprocess.run(['tmux', 'send-keys', blk])

## export -

In [ ]:
#| hide
#| eval: false
from nbdev.doclinks import nbdev_export
nbdev_export()